# 导入库函数

In [1]:
from __future__ import division
from __future__ import print_function
from gravity_gae.evaluation import compute_scores
from gravity_gae.input_data import load_data
from gravity_gae.model import *
from gravity_gae.optimizer import OptimizerAE, OptimizerVAE
from gravity_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错

# 定义超参数

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('dataset', 'Cornell', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'task_2', 'Name of the link prediction task')
''' See section 4.1. of paper for details about tasks:

- task_1: General Directed Link Prediction

- task_2: Biased Negative Samples Directed Link Prediction

- task_3: Bidirectionality Prediction
'''

# Model
flags.DEFINE_string('model', 'source_target_gcn_vae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Variational Graph Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders and inner product decoder

- source_target_gcn_ae: Source-Target Graph Autoencoder, as introduced
                        in section 2.6 of paper, with 2-layer GCN encoder
                        and asymmetric inner product decoder

- source_target_gcn_vae: Source-Target Graph Variational Autoencoder, as
                         introduced in section 2.6, with Gaussian priors,
                         2-layer GCN encoders and asymmetric inner product
 
- gravity_gcn_ae: Gravity-Inspired Graph Autoencoder, as introduced in
                  section 3.3 of paper, with 2-layer GCN encoder and 
                  gravity-inspired asymmetric decoder
 
- gravity_gcn_vae: Gravity-Inspired Graph Variational Autoencoder, as
                   introduced in section 3.4 of paper, with Gaussian 
                   priors, 2-layer GCN encoders and gravity-inspired decoder
'''

# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in GCN')
flags.DEFINE_float('lamb', 1., 'lambda parameter from Gravity AE/VAE models \
                                as introduced in section 3.5 of paper, to \
                                balance mass and proximity terms')
flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 24, 'Number of units in GCN hidden layer.')
flags.DEFINE_integer('dimension', 12, 'Dimension of GCN output: \
- equal to embedding dimension for standard AE/VAE and source-target AE/VAE \
- equal to (embedding dimension - 1) for gravity-inspired AE/VAE, as the \
last dimension captures the "mass" parameter tilde{m}')
flags.DEFINE_boolean('normalize', False, 'Whether to normalize embedding \
                                          vectors of gravity models')
flags.DEFINE_float('epsilon', 0.01, 'Add epsilon to distances computations \
                                       in gravity models, for numerical \
                                       stability')
# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Task 1)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Tasks 1 and 2)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results  at each epoch (for \
                                           Task 1)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

In [3]:
# Lists to collect average results
mean_roc = []
mean_ap = []
mean_time = []


# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")
adj_init, features = load_data(FLAGS.dataset)


# The entire training process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    # Edge Masking: compute Train/Validation/Test set
    if FLAGS.verbose:
        print("Masking test edges...")

    if FLAGS.task == 'task_1':
        # Edge masking for General Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_general_link_prediction(adj_init, FLAGS.prop_test,
                                                FLAGS.prop_val)
    elif FLAGS.task == 'task_2':
        # Edge masking for B.N.S. Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_biased_negative_samples(adj_init, FLAGS.prop_test)
    elif FLAGS.task == 'task_3':
        # Edge masking for Bidirectionality Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_bidirectional_link_prediction(adj_init)
    else:
        raise ValueError('Undefined task!')

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape=())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'source_target_gcn_ae':
        # Source-Target Graph Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelAE(placeholders, num_features,
                                     features_nonzero)
    elif FLAGS.model == 'source_target_gcn_vae':
        # Source-Target Graph Variational Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelVAE(placeholders, num_features,
                                      num_nodes, features_nonzero)
    elif FLAGS.model == 'gravity_gcn_ae':
        # Gravity-Inspired Graph Autoencoder
        model = GravityGCNModelAE(placeholders, num_features,
                                  features_nonzero)
    elif FLAGS.model == 'gravity_gcn_vae':
        # Gravity-Inspired Graph Variational Autoencoder
        model = GravityGCNModelVAE(placeholders, num_features, num_nodes,
                                   features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer (see tkipf/gae original GAE repository for details)
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'source_target_gcn_ae', 'gravity_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'source_target_gcn_vae', 'gravity_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")
    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation (implemented for Task 1 only)
            if FLAGS.validation and FLAGS.task == 'task_1':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = compute_scores(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Compute total running time
    mean_time.append(time.time() - t_start)

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Compute ROC and AP scores on test sets
    roc_score, ap_score = compute_scores(test_edges, test_edges_false, emb)
    # Append to list of scores over all runs
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Loading data...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.59037 time= 0.09900
Epoch: 0002 train_loss= 1.48268 time= 0.00300
Epoch: 0003 train_loss= 1.32836 time= 0.00299
Epoch: 0004 train_loss= 1.25267 time= 0.00198
Epoch: 0005 train_loss= 1.10342 time= 0.00300
Epoch: 0006 train_loss= 1.01947 time= 0.00300
Epoch: 0007 train_loss= 0.99792 time= 0.00300
Epoch: 0008 train_loss= 1.01421 time= 0.00201
Epoch: 0009 train_loss= 1.00004 time= 0.00199
Epoch: 0010 train_loss= 0.95334 time= 0.00301
Epoch: 0011 train_loss= 0.93338 time= 0.00300
Epoch: 0012 train_loss= 0.90715 time= 0.00199
Epoch: 0013 train_loss= 0.89490 time= 0.00301
Epoch: 0014 train_loss= 0.86763 time= 0.00199
Epoch: 0015 train_loss= 0.85214 time= 0.00200
Epoch: 0016 train_loss= 0.83637 time= 0.00301
Epoch: 0017 train_loss= 0.80814 time= 0.00199
Epoch: 0018 train_loss= 0.77800 time= 0.00201
Epoch: 0019 train_loss= 0.74810 time= 0.00300
Epoch: 0020 train_loss= 0.73730 time= 0.00

Training...
Epoch: 0001 train_loss= 1.70957 time= 0.07690
Epoch: 0002 train_loss= 1.47001 time= 0.00600
Epoch: 0003 train_loss= 1.31832 time= 0.00305
Epoch: 0004 train_loss= 1.16944 time= 0.00402
Epoch: 0005 train_loss= 0.98277 time= 0.00300
Epoch: 0006 train_loss= 0.98395 time= 0.00300
Epoch: 0007 train_loss= 0.97416 time= 0.00302
Epoch: 0008 train_loss= 0.97391 time= 0.00198
Epoch: 0009 train_loss= 0.95484 time= 0.00402
Epoch: 0010 train_loss= 0.93706 time= 0.00307
Epoch: 0011 train_loss= 0.90191 time= 0.00291
Epoch: 0012 train_loss= 0.88232 time= 0.00206
Epoch: 0013 train_loss= 0.88808 time= 0.00203
Epoch: 0014 train_loss= 0.85582 time= 0.00299
Epoch: 0015 train_loss= 0.84344 time= 0.00294
Epoch: 0016 train_loss= 0.81655 time= 0.00306
Epoch: 0017 train_loss= 0.78712 time= 0.00304
Epoch: 0018 train_loss= 0.77454 time= 0.00191
Epoch: 0019 train_loss= 0.75219 time= 0.00300
Epoch: 0020 train_loss= 0.74240 time= 0.00307
Epoch: 0021 train_loss= 0.71512 time= 0.00203
Epoch: 0022 train_loss

Training...
Epoch: 0001 train_loss= 1.46985 time= 0.08400
Epoch: 0002 train_loss= 1.39716 time= 0.00300
Epoch: 0003 train_loss= 1.18528 time= 0.00401
Epoch: 0004 train_loss= 1.07842 time= 0.00299
Epoch: 0005 train_loss= 1.05713 time= 0.00401
Epoch: 0006 train_loss= 0.97145 time= 0.00301
Epoch: 0007 train_loss= 1.00385 time= 0.00299
Epoch: 0008 train_loss= 0.98874 time= 0.00302
Epoch: 0009 train_loss= 0.92183 time= 0.00299
Epoch: 0010 train_loss= 0.92590 time= 0.00300
Epoch: 0011 train_loss= 0.91738 time= 0.00201
Epoch: 0012 train_loss= 0.89376 time= 0.00300
Epoch: 0013 train_loss= 0.88765 time= 0.00300
Epoch: 0014 train_loss= 0.86930 time= 0.00300
Epoch: 0015 train_loss= 0.86100 time= 0.00300
Epoch: 0016 train_loss= 0.82357 time= 0.00303
Epoch: 0017 train_loss= 0.81344 time= 0.00196
Epoch: 0018 train_loss= 0.80615 time= 0.00200
Epoch: 0019 train_loss= 0.78152 time= 0.00202
Epoch: 0020 train_loss= 0.77047 time= 0.00187
Epoch: 0021 train_loss= 0.75083 time= 0.00300
Epoch: 0022 train_loss

Training...
Epoch: 0001 train_loss= 1.66052 time= 0.10915
Epoch: 0002 train_loss= 1.48577 time= 0.00315
Epoch: 0003 train_loss= 1.22197 time= 0.00289
Epoch: 0004 train_loss= 1.11354 time= 0.00300
Epoch: 0005 train_loss= 1.00070 time= 0.00300
Epoch: 0006 train_loss= 0.98985 time= 0.00301
Epoch: 0007 train_loss= 0.99240 time= 0.00400
Epoch: 0008 train_loss= 0.95419 time= 0.00295
Epoch: 0009 train_loss= 0.96476 time= 0.00200
Epoch: 0010 train_loss= 0.92755 time= 0.00205
Epoch: 0011 train_loss= 0.90665 time= 0.00301
Epoch: 0012 train_loss= 0.89513 time= 0.00297
Epoch: 0013 train_loss= 0.88103 time= 0.00198
Epoch: 0014 train_loss= 0.86608 time= 0.00205
Epoch: 0015 train_loss= 0.85213 time= 0.00195
Epoch: 0016 train_loss= 0.82990 time= 0.00200
Epoch: 0017 train_loss= 0.81650 time= 0.00300
Epoch: 0018 train_loss= 0.79070 time= 0.00304
Epoch: 0019 train_loss= 0.76098 time= 0.00198
Epoch: 0020 train_loss= 0.74331 time= 0.00302
Epoch: 0021 train_loss= 0.72143 time= 0.00312
Epoch: 0022 train_loss

Epoch: 0200 train_loss= 0.35921 time= 0.00198
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.62545 time= 0.10201
Epoch: 0002 train_loss= 1.37648 time= 0.00301
Epoch: 0003 train_loss= 1.14027 time= 0.00326
Epoch: 0004 train_loss= 1.05525 time= 0.00275
Epoch: 0005 train_loss= 1.04625 time= 0.00299
Epoch: 0006 train_loss= 1.01222 time= 0.00301
Epoch: 0007 train_loss= 0.97948 time= 0.00302
Epoch: 0008 train_loss= 1.00138 time= 0.00399
Epoch: 0009 train_loss= 0.94998 time= 0.00298
Epoch: 0010 train_loss= 0.95381 time= 0.00200
Epoch: 0011 train_loss= 0.91587 time= 0.00301
Epoch: 0012 train_loss= 0.87648 time= 0.00401
Epoch: 0013 train_loss= 0.85654 time= 0.00299
Epoch: 0014 train_loss= 0.84257 time= 0.00299
Epoch: 0015 train_loss= 0.83379 time= 0.00301
Epoch: 0016 train_loss= 0.80959 time= 0.00303
Epoch: 0017 train_loss= 0.80330 time= 0.00397
Epoch: 0018 train_loss= 0.76767 time= 0.00300
Epoch: 0019 train_loss= 0.75888 time= 0.0

...

Epoch: 0198 train_loss= 0.36741 time= 0.00303
Epoch: 0199 train_loss= 0.36957 time= 0.00296
Epoch: 0200 train_loss= 0.37275 time= 0.00354
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.69254 time= 0.12404
Epoch: 0002 train_loss= 1.44470 time= 0.00512
Epoch: 0003 train_loss= 1.09513 time= 0.00200
Epoch: 0004 train_loss= 1.05853 time= 0.00401
Epoch: 0005 train_loss= 1.03058 time= 0.00408
Epoch: 0006 train_loss= 0.96788 time= 0.00394
Epoch: 0007 train_loss= 1.07361 time= 0.00302
Epoch: 0008 train_loss= 0.95481 time= 0.00401
Epoch: 0009 train_loss= 0.95870 time= 0.00295
Epoch: 0010 train_loss= 0.96423 time= 0.00304
Epoch: 0011 train_loss= 0.92634 time= 0.00300
Epoch: 0012 train_loss= 0.88378 time= 0.00296
Epoch: 0013 train_loss= 0.85643 time= 0.00304
Epoch: 0014 train_loss= 0.84834 time= 0.00305
Epoch: 0015 train_loss= 0.83721 time= 0.00210
Epoch: 0016 train_loss= 0.82087 time= 0.00307
Epoch: 0017 train_loss= 0.80434 time= 0.0

Epoch: 0188 train_loss= 0.36529 time= 0.00273
Epoch: 0189 train_loss= 0.37413 time= 0.00303
Epoch: 0190 train_loss= 0.38690 time= 0.00301
Epoch: 0191 train_loss= 0.37127 time= 0.00457
Epoch: 0192 train_loss= 0.37145 time= 0.00200
Epoch: 0193 train_loss= 0.37992 time= 0.00330
Epoch: 0194 train_loss= 0.38159 time= 0.00308
Epoch: 0195 train_loss= 0.38346 time= 0.00299
Epoch: 0196 train_loss= 0.37669 time= 0.00444
Epoch: 0197 train_loss= 0.37545 time= 0.00253
Epoch: 0198 train_loss= 0.37065 time= 0.00366
Epoch: 0199 train_loss= 0.36394 time= 0.00243
Epoch: 0200 train_loss= 0.37671 time= 0.00414
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.64947 time= 0.12905
Epoch: 0002 train_loss= 1.42678 time= 0.00399
Epoch: 0003 train_loss= 1.25430 time= 0.00303
Epoch: 0004 train_loss= 1.07809 time= 0.00396
Epoch: 0005 train_loss= 0.99972 time= 0.00199
Epoch: 0006 train_loss= 0.99962 time= 0.00298
Epoch: 0007 train_loss= 1.01709 time= 0.0

Epoch: 0184 train_loss= 0.35895 time= 0.00258
Epoch: 0185 train_loss= 0.35745 time= 0.00302
Epoch: 0186 train_loss= 0.35651 time= 0.00302
Epoch: 0187 train_loss= 0.35113 time= 0.00302
Epoch: 0188 train_loss= 0.35232 time= 0.00536
Epoch: 0189 train_loss= 0.35709 time= 0.00355
Epoch: 0190 train_loss= 0.37933 time= 0.00302
Epoch: 0191 train_loss= 0.36301 time= 0.00306
Epoch: 0192 train_loss= 0.35972 time= 0.00293
Epoch: 0193 train_loss= 0.34592 time= 0.00305
Epoch: 0194 train_loss= 0.35119 time= 0.00395
Epoch: 0195 train_loss= 0.36784 time= 0.00312
Epoch: 0196 train_loss= 0.36147 time= 0.00296
Epoch: 0197 train_loss= 0.36414 time= 0.00309
Epoch: 0198 train_loss= 0.35552 time= 0.00199
Epoch: 0199 train_loss= 0.36949 time= 0.00296
Epoch: 0200 train_loss= 0.36966 time= 0.00195
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.63792 time= 0.12972
Epoch: 0002 train_loss= 1.51369 time= 0.00295
Epoch: 0003 train_loss= 1.12235 time= 0.0

Epoch: 0187 train_loss= 0.35548 time= 0.00309
Epoch: 0188 train_loss= 0.35832 time= 0.00305
Epoch: 0189 train_loss= 0.36165 time= 0.00296
Epoch: 0190 train_loss= 0.35476 time= 0.00302
Epoch: 0191 train_loss= 0.34551 time= 0.00400
Epoch: 0192 train_loss= 0.35096 time= 0.00294
Epoch: 0193 train_loss= 0.34295 time= 0.00306
Epoch: 0194 train_loss= 0.34676 time= 0.00294
Epoch: 0195 train_loss= 0.36761 time= 0.00405
Epoch: 0196 train_loss= 0.37045 time= 0.00297
Epoch: 0197 train_loss= 0.35916 time= 0.00299
Epoch: 0198 train_loss= 0.35484 time= 0.00502
Epoch: 0199 train_loss= 0.37501 time= 0.00198
Epoch: 0200 train_loss= 0.34859 time= 0.00206
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.64491 time= 0.14296
Epoch: 0002 train_loss= 1.43541 time= 0.00409
Epoch: 0003 train_loss= 1.27546 time= 0.00444
Epoch: 0004 train_loss= 1.01912 time= 0.00340
Epoch: 0005 train_loss= 1.03004 time= 0.00201
Epoch: 0006 train_loss= 0.99149 time= 0.0

Epoch: 0183 train_loss= 0.38324 time= 0.00200
Epoch: 0184 train_loss= 0.36547 time= 0.00307
Epoch: 0185 train_loss= 0.39112 time= 0.00293
Epoch: 0186 train_loss= 0.37561 time= 0.00201
Epoch: 0187 train_loss= 0.37695 time= 0.00307
Epoch: 0188 train_loss= 0.38244 time= 0.00293
Epoch: 0189 train_loss= 0.37398 time= 0.00304
Epoch: 0190 train_loss= 0.36212 time= 0.00301
Epoch: 0191 train_loss= 0.36206 time= 0.00200
Epoch: 0192 train_loss= 0.37533 time= 0.00281
Epoch: 0193 train_loss= 0.36753 time= 0.00204
Epoch: 0194 train_loss= 0.37929 time= 0.00198
Epoch: 0195 train_loss= 0.37547 time= 0.00301
Epoch: 0196 train_loss= 0.36774 time= 0.00303
Epoch: 0197 train_loss= 0.36373 time= 0.00198
Epoch: 0198 train_loss= 0.36903 time= 0.00195
Epoch: 0199 train_loss= 0.37865 time= 0.00404
Epoch: 0200 train_loss= 0.37140 time= 0.00195
Testing model...

Test results for source_target_gcn_vae model on Cornell on task_2 
 ___________________________________________________

AUC scores
 [0.7568, 0.6192, 0.68